<h1>Считывание данных</h1>

In [5]:
from time import time
from IPython import get_ipython
from IPython.display import Audio, display


class Beeper:

    def __init__(self, threshold, **audio_kwargs):
        self.threshold = threshold
        self.start_time = None    # time in sec, or None
        self.audio = audio_kwargs

    def pre_execute(self):
        if not self.start_time:
            self.start_time = time()

    def post_execute(self):
        end_time = time()
        if self.start_time and end_time - self.start_time > self.threshold:
            audio = Audio(**self.audio, autoplay=True)
            display(audio)
        self.start_time = None
    
    
    def exception(self):
        end_time = time()
        if self.start_time and end_time - self.start_time > self.threshold:
            audio = Audio(**self.audio, autoplay=True)
            display(audio)
        self.start_time = None


beeper = Beeper(5, url='./alert.wav')

ipython = get_ipython()
ipython.events.register('pre_execute', beeper.pre_execute)
ipython.events.register('post_execute', beeper.post_execute)

def play_sound(self, etype, value, tb, tb_offset=None):
    self.showtraceback((etype, value, tb), tb_offset=tb_offset)
    display(Audio(url='./horn.wav', autoplay=True))
    
get_ipython().set_custom_exc((Exception,), play_sound)

In [ ]:
# %matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file = "aes_data.xlsx"

data = pd.read_excel(file)

df = data.drop([0,1,2], axis=0)
cols_fillna = df.columns.values
for col in cols_fillna[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    
pump_in_cols = df.columns.values[1:6]
pump_out_cols = df.columns.values[6:11]
pump_mass_cols = df.columns.values[11:16]
pump_temp_col = df.columns.values[16]
pump_amper_cols = df.columns.values[17:22]
pump_revs_cols = df.columns.values[22:27]
pump_volt_cols = df.columns.values[27:31]
pump_turbine_col = df.columns.values[31]
pump_reactor_col = df.columns.values[32]


# df[cols_fillna[-1]][df[cols_fillna[-1]].isnull()]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data.head()


<h1>Первичная фильтрация данных</h1>
<p>убираем отрицательные или слишком большие значения</p>
<p>+ графики массовой подачи для всех насосов</p>


In [ ]:
df[pump_in_cols[2]] = df[pump_in_cols[2]].apply(lambda x: x if x<=8 else x-x//1)   


for col in df.columns.values[1:]:
    df[col] = df[col].apply(lambda x: 0 if x<0 else x)

for col in pump_mass_cols:
    df[col] = df[col].apply(lambda x: x if x<510 else 0)    
    
plt.rcParams["figure.figsize"] = (8,5)

df.plot(use_index=True, y=pump_mass_cols, title='Показания ПЭНов')
plt.title('Показания ПЭНов')
plt.ylabel('Массовая подача, кг/с', fontsize=15)
plt.legend(loc='lower right')
plt.show()
# ax.show()


<h1>Расчеты КПД</h1>

In [ ]:
from iapws import IAPWS97, IAPWS95

def eff_func(row, pump_num):
    p1 = df[f'10LAB{pump_num}CP002_XQ01'][row]
    p2 = df[f'10LAB{pump_num}CP003_XQ01'][row]
    G = df[f'10LAB{pump_num}CF001_XQ01'][row]
    t = df["10LAB00FT902_XQ01"][row]
    I = df[f"10LAC{pump_num}CE021_XQ01"][row]
    U = df["10BBA00CE010_XQ01"][row]
    
    if all([(p2-p1)*G,U*I, t, p2, p1]):
        try:
            rho = IAPWS97(P=(p1+p2)/2, T=t+273).rho
        except:
            print(p1,p2,G,t, pump_num)
            raise Exception('here')
        ef = (p2-p1)*1000*G/(U*I*rho)/np.sqrt(3)
        ef = 0 if ef < 0 or ef >= 1 else ef
        
        return ef / np.sqrt(3)
        
    else:
        return 0


pump_1_eff = df.apply(lambda x: eff_func(x.name, 11), axis=1)
pump_2_eff = df.apply(lambda x: eff_func(x.name, 12), axis=1)
pump_3_eff = df.apply(lambda x: eff_func(x.name, 13), axis=1)
pump_4_eff = df.apply(lambda x: eff_func(x.name, 14), axis=1)
pump_5_eff = df.apply(lambda x: eff_func(x.name, 15), axis=1)

pump_eff_data = {
    'pump_1_eff':pump_1_eff,
    'pump_2_eff':pump_2_eff,
    'pump_3_eff':pump_3_eff,
    'pump_4_eff':pump_4_eff,
    'pump_5_eff':pump_5_eff,
}



In [ ]:
fig1 = plt.figure(figsize=(8, 5))
ax1 = fig1.add_subplot(111)

plot = pump_eff_data['pump_1_eff'].plot(title="Показания ПЭН-I", axes=ax1)
plt.ylabel("КПД насоса", fontsize=12)

<h1>Разделение на сегменты</h1>

In [ ]:
def divide_to_segments_by_eff(column, limiter):
    segm_start = None
    segm_end = None

    segments = []

    for ind,item in pump_eff_data[column].iteritems():
        if item>limiter:
            if segm_start is None:
                segm_start = ind
        else:
            if segm_start is not None and ind != segm_start:
                segm_end = ind

        if segm_start is not None and segm_end is not None:
            segments.append((segm_start-2,segm_end-3))
            segm_start, segm_end = None, None
    
    return segments

def divide_to_segments(column, limiter):
    segm_start = None
    segm_end = None

    segments = []

    for ind,item in df[column].iteritems():
        if item>limiter:
            if segm_start is None:
                segm_start = ind
        else:
            if segm_start is not None and ind != segm_start:
                segm_end = ind

        if segm_start is not None and segm_end is not None:
            segments.append((segm_start-2,segm_end-3))
            segm_start, segm_end = None, None
    
    return segments
        
# segments1 = divide_to_segments(pump_mass_cols[0], 350)
# segments2 = divide_to_segments(pump_mass_cols[1], 350)
# segments3 = divide_to_segments(pump_mass_cols[2], 350)
# segments4 = divide_to_segments(pump_mass_cols[3], 350)
# segments5 = divide_to_segments(pump_mass_cols[4], 350)

segments1 = divide_to_segments_by_eff('pump_1_eff', 0.1)
segments2 = divide_to_segments_by_eff('pump_2_eff', 0.1)
segments3 = divide_to_segments_by_eff('pump_3_eff', 0.1)
segments4 = divide_to_segments_by_eff('pump_4_eff', 0.1)
segments5 = divide_to_segments_by_eff('pump_5_eff', 0.1)


pump_mass_1_filtred = pd.concat([df[pump_mass_cols[0]][i[0]-2:i[1]-3] for i in segments1],  ignore_index=True)

filtred_data = {
    'pump_1_in': pd.concat([df[pump_in_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_in': pd.concat([df[pump_in_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_in': pd.concat([df[pump_in_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_in': pd.concat([df[pump_in_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_in': pd.concat([df[pump_in_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
    
    'pump_1_out': pd.concat([df[pump_out_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_out': pd.concat([df[pump_out_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_out': pd.concat([df[pump_out_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_out': pd.concat([df[pump_out_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_out': pd.concat([df[pump_out_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
    
    
    'pump_1_mass': pd.concat([df[pump_mass_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_mass': pd.concat([df[pump_mass_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_mass': pd.concat([df[pump_mass_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_mass': pd.concat([df[pump_mass_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_mass': pd.concat([df[pump_mass_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
 
    'pump_1_eff': pd.concat([pump_1_eff[i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_eff': pd.concat([pump_2_eff[i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_eff': pd.concat([pump_3_eff[i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_eff': pd.concat([pump_4_eff[i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_eff': pd.concat([pump_5_eff[i[0]:i[1]] for i in segments5],  ignore_index=True),
    
    'pump_1_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments5],  ignore_index=True),
    
    
    'pump_1_amper': pd.concat([df[pump_amper_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_amper': pd.concat([df[pump_amper_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_amper': pd.concat([df[pump_amper_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_amper': pd.concat([df[pump_amper_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_amper': pd.concat([df[pump_amper_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),

    'pump_1_revs': pd.concat([df[pump_revs_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_revs': pd.concat([df[pump_revs_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_revs': pd.concat([df[pump_revs_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_revs': pd.concat([df[pump_revs_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_revs': pd.concat([df[pump_revs_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
    
#     'pump_1_volt': pd.concat([df[pump_volt_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
#     'pump_2_volt': pd.concat([df[pump_volt_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
#     'pump_3_volt': pd.concat([df[pump_volt_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
#     'pump_4_volt': pd.concat([df[pump_volt_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
#     'pump_5_volt': pd.concat([df[pump_volt_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
}
    
    
filtred_df = pd.DataFrame(filtred_data)

# for i in segments1:
#     print('--',pump1_eff[i[0]], pump1_eff[i[1]])
#     pump1_eff[i[0]:i[1]].plot()



In [ ]:
fig2 = plt.figure(figsize=(8, 5))
ax2 = fig2.add_subplot(111)

filtred_data['pump_1_mass'].plot(title="Показания ПЭН-I", axes=ax2, label='Данные после фильтрации')

plt.ylabel("Массовый расход, кг/c", fontsize=15)

rolled = pump_mass_1_filtred.rolling(50).mean()
rolled.plot(linewidth=3, axes=ax2, label='Сглаженные данные')

ax2.legend(loc='lower right')


<h1>Сглаживание</h1>

In [ ]:
def smooth(data, rolling=10, min_periods=1):
    f_max = data.max()
    f_norm = data/f_max
    return f_norm.rolling(rolling, min_periods).mean()

smoothed_data = {
    'pump_1_in':smooth(filtred_data['pump_1_in']),
    'pump_2_in':smooth(filtred_data['pump_2_in']),
    'pump_3_in':smooth(filtred_data['pump_3_in']),
    'pump_4_in':smooth(filtred_data['pump_4_in']),
    'pump_5_in':smooth(filtred_data['pump_5_in']),
    
    
    'pump_1_out':smooth(filtred_data['pump_1_out']),
    'pump_2_out':smooth(filtred_data['pump_2_out']),
    'pump_3_out':smooth(filtred_data['pump_3_out']),
    'pump_4_out':smooth(filtred_data['pump_4_out']),
    'pump_5_out':smooth(filtred_data['pump_5_out']),
    
    
    'pump_1_mass':smooth(filtred_data['pump_1_mass']),
    'pump_2_mass':smooth(filtred_data['pump_2_mass']),
    'pump_3_mass':smooth(filtred_data['pump_3_mass']),
    'pump_4_mass':smooth(filtred_data['pump_4_mass']),
    'pump_5_mass':smooth(filtred_data['pump_5_mass']),
    
    'pump_1_eff':smooth(filtred_data['pump_1_eff']),
    'pump_2_eff':smooth(filtred_data['pump_2_eff']),
    'pump_3_eff':smooth(filtred_data['pump_3_eff']),
    'pump_4_eff':smooth(filtred_data['pump_4_eff']),
    'pump_5_eff':smooth(filtred_data['pump_5_eff']),
    
    'pump_1_temp':smooth(filtred_data['pump_1_temp'], rolling=10),
    'pump_2_temp':smooth(filtred_data['pump_2_temp'], rolling=10),
    'pump_3_temp':smooth(filtred_data['pump_3_temp'], rolling=10),
    'pump_4_temp':smooth(filtred_data['pump_4_temp'], rolling=10),
    'pump_5_temp':smooth(filtred_data['pump_5_temp'], rolling=10),
    
    'pump_1_revs':smooth(filtred_data['pump_1_revs']),
    'pump_2_revs':smooth(filtred_data['pump_2_revs']),
    'pump_3_revs':smooth(filtred_data['pump_3_revs']),
    'pump_4_revs':smooth(filtred_data['pump_4_revs']),
    'pump_5_revs':smooth(filtred_data['pump_5_revs']),
    
    'pump_1_amper':smooth(filtred_data['pump_1_amper']),
    'pump_2_amper':smooth(filtred_data['pump_2_amper']),
    'pump_3_amper':smooth(filtred_data['pump_3_amper']),
    'pump_4_amper':smooth(filtred_data['pump_4_amper']),
    'pump_5_amper':smooth(filtred_data['pump_5_amper']),
}



In [ ]:

fig3 = plt.figure(figsize=(8, 5))
ax3 = fig3.add_subplot(111)


pump_mass_1_filtred
pump_mass_1_max = filtred_data['pump_1_mass'].max()

pump_mass_1_norm = filtred_data['pump_1_mass'] / pump_mass_1_max
pump_mass_1_norm.plot(title="Показания ПЭН-I", label="Нормализованный массовый расход", axes=ax3)
# plt.ylabel("Массовый расход", fontsize=15)

rolled_mass_1 = pump_mass_1_norm.rolling(20, min_periods=1).mean()
rolled_mass_1.plot(label='Сглаженные данные', linewidth=3, axes=ax3)
plt.ylabel("Массовый расход, кг/c", fontsize=12)
ax3.legend(loc='lower right')


# smoothed_data['pump_5_eff'].plot(axes=ax3, title='Сглаживание данных')
# (filtred_data['pump_5_temp']/filtred_data['pump_5_temp'].max()).plot(axes=ax3)


In [ ]:
fig4 = plt.figure(figsize=(8, 5))
ax4 = fig4.add_subplot(111)

(filtred_data['pump_1_in']/filtred_data['pump_1_in'].max()).plot(title='Показания ПЭН-I', label="Нормализованное давление на всасе",axes=ax4)
smoothed_data['pump_1_in'].plot(linewidth=2, label="Сглаженные данные", axes=ax4)
plt.ylabel("Давление на всасе", fontsize=12)
ax4.legend(loc="lower right")


<h1>Центрирование</h1>

In [ ]:
norm_pump_1_in = (smoothed_data['pump_1_in']-smoothed_data['pump_1_in'].mean())/smoothed_data['pump_1_in']
norm_pump_1_eff = (smoothed_data['pump_1_eff']-smoothed_data['pump_1_eff'].mean())/smoothed_data['pump_1_eff']

norm_pump_2_in = (smoothed_data['pump_2_in']-smoothed_data['pump_2_in'].mean())/smoothed_data['pump_2_in']
norm_pump_2_eff = (smoothed_data['pump_2_eff']-smoothed_data['pump_2_eff'].mean())/smoothed_data['pump_2_eff']

norm_pump_3_in = (smoothed_data['pump_3_in']-smoothed_data['pump_3_in'].mean())/smoothed_data['pump_3_in']
norm_pump_3_eff = (smoothed_data['pump_3_eff']-smoothed_data['pump_3_eff'].mean())/smoothed_data['pump_3_eff']

centred_data = {
    'pump_1_in': (smoothed_data['pump_1_in']-smoothed_data['pump_1_in'].mean())/smoothed_data['pump_1_in'],
    'pump_2_in': (smoothed_data['pump_2_in']-smoothed_data['pump_2_in'].mean())/smoothed_data['pump_2_in'],
    'pump_3_in': (smoothed_data['pump_3_in']-smoothed_data['pump_3_in'].mean())/smoothed_data['pump_3_in'],
    'pump_4_in': (smoothed_data['pump_4_in']-smoothed_data['pump_4_in'].mean())/smoothed_data['pump_4_in'],
    'pump_5_in': (smoothed_data['pump_5_in']-smoothed_data['pump_5_in'].mean())/smoothed_data['pump_5_in'],

    'pump_1_out': (smoothed_data['pump_1_out']-smoothed_data['pump_1_out'].mean())/smoothed_data['pump_1_out'],
    'pump_2_out': (smoothed_data['pump_2_out']-smoothed_data['pump_2_out'].mean())/smoothed_data['pump_2_out'],
    'pump_3_out': (smoothed_data['pump_3_out']-smoothed_data['pump_3_out'].mean())/smoothed_data['pump_3_out'],
    'pump_4_out': (smoothed_data['pump_4_out']-smoothed_data['pump_4_out'].mean())/smoothed_data['pump_4_out'],
    'pump_5_out': (smoothed_data['pump_5_out']-smoothed_data['pump_5_out'].mean())/smoothed_data['pump_5_out'],

    'pump_1_eff': (smoothed_data['pump_1_eff']-smoothed_data['pump_1_eff'].mean())/smoothed_data['pump_1_eff'],
    'pump_2_eff': (smoothed_data['pump_2_eff']-smoothed_data['pump_2_eff'].mean())/smoothed_data['pump_2_eff'],
    'pump_3_eff': (smoothed_data['pump_3_eff']-smoothed_data['pump_3_eff'].mean())/smoothed_data['pump_3_eff'],
    'pump_4_eff': (smoothed_data['pump_4_eff']-smoothed_data['pump_4_eff'].mean())/smoothed_data['pump_4_eff'],
    'pump_5_eff': (smoothed_data['pump_5_eff']-smoothed_data['pump_5_eff'].mean())/smoothed_data['pump_5_eff'],
    
    'pump_1_temp': (smoothed_data['pump_1_temp']-smoothed_data['pump_1_temp'].mean())/smoothed_data['pump_1_temp'],
    'pump_2_temp': (smoothed_data['pump_2_temp']-smoothed_data['pump_2_temp'].mean())/smoothed_data['pump_2_temp'],
    'pump_3_temp': (smoothed_data['pump_3_temp']-smoothed_data['pump_3_temp'].mean())/smoothed_data['pump_3_temp'],
    'pump_4_temp': (smoothed_data['pump_4_temp']-smoothed_data['pump_4_temp'].mean())/smoothed_data['pump_4_temp'],
    'pump_5_temp': (smoothed_data['pump_5_temp']-smoothed_data['pump_5_temp'].mean())/smoothed_data['pump_5_temp'],

    'pump_1_revs': (smoothed_data['pump_1_revs']-smoothed_data['pump_1_revs'].mean())/smoothed_data['pump_1_revs'],
    'pump_2_revs': (smoothed_data['pump_2_revs']-smoothed_data['pump_2_revs'].mean())/smoothed_data['pump_2_revs'],
    'pump_3_revs': (smoothed_data['pump_3_revs']-smoothed_data['pump_3_revs'].mean())/smoothed_data['pump_3_revs'],
    'pump_4_revs': (smoothed_data['pump_4_revs']-smoothed_data['pump_4_revs'].mean())/smoothed_data['pump_4_revs'],
    'pump_5_revs': (smoothed_data['pump_5_revs']-smoothed_data['pump_5_revs'].mean())/smoothed_data['pump_5_revs'],

    'pump_1_amper': (smoothed_data['pump_1_amper']-smoothed_data['pump_1_amper'].mean())/smoothed_data['pump_1_amper'],
    'pump_2_amper': (smoothed_data['pump_2_amper']-smoothed_data['pump_2_amper'].mean())/smoothed_data['pump_2_amper'],
    'pump_3_amper': (smoothed_data['pump_3_amper']-smoothed_data['pump_3_amper'].mean())/smoothed_data['pump_3_amper'],
    'pump_4_amper': (smoothed_data['pump_4_amper']-smoothed_data['pump_4_amper'].mean())/smoothed_data['pump_4_amper'],
    'pump_5_amper': (smoothed_data['pump_5_amper']-smoothed_data['pump_5_amper'].mean())/smoothed_data['pump_5_amper'],
}

corr_coefs = [
    # p_in -- p_eff
    np.corrcoef(centred_data['pump_1_in'].to_numpy(), centred_data['pump_1_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_2_in'].to_numpy(), centred_data['pump_2_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_3_in'].to_numpy(), centred_data['pump_3_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_4_in'].to_numpy(), centred_data['pump_4_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_5_in'].to_numpy(), centred_data['pump_5_eff'].to_numpy()),

    # p_out -- p_eff
    np.corrcoef(centred_data['pump_1_out'].to_numpy(), centred_data['pump_1_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_2_out'].to_numpy(), centred_data['pump_2_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_3_out'].to_numpy(), centred_data['pump_3_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_4_out'].to_numpy(), centred_data['pump_4_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_5_out'].to_numpy(), centred_data['pump_5_eff'].to_numpy()),

    
    # p_temp -- p_eff
    np.corrcoef(centred_data['pump_1_temp'].to_numpy(), centred_data['pump_1_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_2_temp'].to_numpy(), centred_data['pump_2_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_3_temp'].to_numpy(), centred_data['pump_3_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_4_temp'].to_numpy(), centred_data['pump_4_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_5_temp'].to_numpy(), centred_data['pump_5_eff'].to_numpy()),    
    
    # p_I -- p_eff
    np.corrcoef(centred_data['pump_1_amper'].to_numpy(), centred_data['pump_1_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_2_amper'].to_numpy(), centred_data['pump_2_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_3_amper'].to_numpy(), centred_data['pump_3_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_4_amper'].to_numpy(), centred_data['pump_4_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_5_amper'].to_numpy(), centred_data['pump_5_eff'].to_numpy()),    
]


for ind, coef in enumerate(corr_coefs):
    if ind % 5 == 0:
        print("---------------")
    print(coef[0][1])


<h1>Линейная регрессия</h1>
<p>Одномерная для Pin/out-КПД, Tпв-КПД</p>
<p>Многомернаяя для Pin+Pout-КПД

<h5>Давление - КПД</h5>

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

model1 = linear_model.LinearRegression()

x_test = centred_data['pump_5_out'].values.reshape(-1,1)
y_test = centred_data['pump_5_eff'].values.reshape(-1,1)


model1.fit(x_test, y_test)

y_pred = model1.predict(x_test)

fig51 = plt.figure(figsize=(8, 5))
ax51 = fig51.add_subplot(111)

ax51.scatter(x_test, y_test, s=3, label='Нормализованные данные (ПЭН-I)')
ax51.plot(x_test, y_pred, color='orange', linewidth=3, label='Линия регрессии (ПЭН-I)')
ax51.legend(loc="lower right")
ax51.set_ylabel("КПД насоса")
ax51.set_xlabel("Давление на напоре")



print("R^2:", model1.score(x_test, y_test))
print("MAE: {}".format(np.abs(y_test-y_pred).mean()))
print("RMSE: {}".format(np.sqrt(((y_test-y_pred)**2).mean())))
print(f'y = {round(model1.coef_[0][0],4)} + {round(model1.intercept_[0],6)}')

In [ ]:
from sklearn.preprocessing import PolynomialFeatures 
nb_degree = 4
poly_reg = PolynomialFeatures(degree = nb_degree)

x = filtred_data['pump_2_out'].to_numpy()
y = filtred_data['pump_2_eff'].to_numpy()


y = y[x.argsort()]
x = x[x.argsort()]

x = x[:, np.newaxis]
y = y[:, np.newaxis]


X_TRANSF = poly_reg.fit_transform(x)

poly_reg.fit(X_TRANSF, y)

lin_reg = linear_model.LinearRegression()
lin_reg.fit(X_TRANSF, y)

fig511 = plt.figure(figsize=(8, 5))
ax511 = fig511.add_subplot(111)

ax511.scatter(x, y)
ax511.plot(x,lin_reg.predict(X_TRANSF), color='orange')


<h5>Температура пит. воды - КПД </h5>

In [ ]:
model2 = linear_model.LinearRegression()

x_test = centred_data['pump_5_temp'].values.reshape(-1,1)
y_test = centred_data['pump_5_eff'].values.reshape(-1,1)


model2.fit(x_test, y_test)

y_pred = model2.predict(x_test)

fig52 = plt.figure(figsize=(8, 5))
ax52 = fig52.add_subplot(111)

ax52.scatter(x_test, y_test, s=3, label='Нормализованные данные (ПЭН-I)')
ax52.plot(x_test, y_pred, color='orange', linewidth=3, label='Линия регрессии (ПЭН-I)')
ax52.legend(loc="lower right")
ax52.set_ylabel("КПД насоса")
ax52.set_xlabel("Темп. пит. воды")

print("R^2:", model2.score(x_test, y_test))
print("MAE: {}".format(np.abs(y_test-y_pred).mean()))
print("RMSE: {}".format(np.sqrt(((y_test-y_pred)**2).mean())))
print(f'y = {round(model2.coef_[0][0],4)} + {round(model2.intercept_[0],6)}')


<h5>Многомерная регрессия</h5>

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

model3 = linear_model.LinearRegression()

reg_df = pd.DataFrame(
    {
        'pump_1_in':filtred_df['pump_1_in'],
        'pump_1_out':filtred_df['pump_1_out'],
        'pump_1_eff':filtred_df['pump_1_eff'],
    }
)

x_test = reg_df[['pump_1_in', 'pump_1_out']]
y_test = reg_df['pump_1_eff']

model3.fit(x_test, y_test)

y_pred = model3.predict(x_test)


score = model3.score(x_test, y_test)

reg_df = pd.DataFrame(
    {
        'pump_1_in':filtred_df['pump_1_in'],
        'pump_1_out':filtred_df['pump_1_out'],
#         'pump_1_eff':filtred_df['pump_1_eff'],
    }
)

x = reg_df[['pump_1_in', 'pump_1_out']].values
y = filtred_df['pump_1_eff'].to_numpy()


fig6 = plt.figure(figsize=(25, 10))
ax6 = fig6.add_subplot(131, projection='3d')

ax6.scatter(x[:,0], x[:,1], y, s=3)
ax6.scatter(x[:,0], x[:,1], y_pred, s=8)

print("R^2:", score)
print("MAE: {}".format(np.abs(y_test-y_pred).mean()))
print("RMSE: {}".format(np.sqrt(((y_test-y_pred)**2).mean())))


coefs = model3.coef_
intercept = model3.intercept_

import math
n = (x[:,0].max()-x[:,0].min())/0.01
m = (x[:,1].max()-x[:,1].min())/0.01

n = math.ceil(n)
m = math.ceil(m)

# xs = np.tile(np.arange(x[:,0].min(),x[:,0].max(), 0.01), (n,1))
# ys = np.tile(np.arange(x[:,1].min(),x[:,1].max(), 0.01), (m,1)).T

xs = np.linspace(x[:,0].min(), x[:,0].max(), 50)
ys = np.linspace(x[:,1].min(), x[:,1].max(), 50)

xs, ys = np.meshgrid(xs,ys)

zs = xs*coefs[0]+ys*coefs[1]+intercept

ax6.plot_surface(xs,ys,zs, alpha=0.5)
plt.show()

<h5>Попарные коэф. корреляции</h5>

In [ ]:
# # df.rename(columns={
# #     pump_in_cols[0]: "pump_1_in",
# #     pump_in_cols[1]: "pump_2_in",
# #     pump_in_cols[2]: "pump_3_in",
# #     pump_in_cols[3]: "pump_4_in",
# #     pump_in_cols[4]: "pump_5_in",
    
# #     pump_out_cols[0]: "pump_1_out",
# #     pump_out_cols[1]: "pump_2_out",
# #     pump_out_cols[2]: "pump_3_out",
# #     pump_out_cols[3]: "pump_4_out",
# #     pump_out_cols[4]: "pump_5_out",
    
# #     pump_mass_cols[0]: "pump_1_mass",
# #     pump_mass_cols[1]: "pump_2_mass",
# #     pump_mass_cols[2]: "pump_3_mass",
# #     pump_mass_cols[3]: "pump_4_mass",
# #     pump_mass_cols[4]: "pump_5_mass",
    
# #     pump_temp_col: "temp",
    
# #     pump_amper_cols[0]: "pump_1_amper",
# #     pump_amper_cols[1]: "pump_2_amper",
# #     pump_amper_cols[2]: "pump_3_amper",
# #     pump_amper_cols[3]: "pump_4_amper",
# #     pump_amper_cols[4]: "pump_5_amper",
    
# #     pump_revs_cols[0]: "pump_1_revs",
# #     pump_revs_cols[1]: "pump_2_revs",
# #     pump_revs_cols[2]: "pump_3_revs",
# #     pump_revs_cols[3]: "pump_4_revs",
# #     pump_revs_cols[4]: "pump_5_revs",
    
# #     pump_volt_cols[0]: "pump_1_volt",
# #     pump_volt_cols[1]: "pump_2_volt",
# #     pump_volt_cols[2]: "pump_3_volt",
# #     pump_volt_cols[3]: "pump_4_volt",
    
# #     pump_turbine_col: "pump_turbine",
# #     pump_reactor_col: "pump_reactor",
# # }, inplace = True)

# df.rename(columns={
#     pump_in_cols[0]: "ПЭН-1 Pнап",
#     pump_in_cols[1]: "ПЭН-2 Pнап",
#     pump_in_cols[2]: "ПЭН-3 Pнап",
#     pump_in_cols[3]: "ПЭН-4 Pнап",
#     pump_in_cols[4]: "ПЭН-5 Pнап",
    
#     pump_out_cols[0]: "ПЭН-1 Pвсас",
#     pump_out_cols[1]: "ПЭН-2 Pвсас",
#     pump_out_cols[2]: "ПЭН-3 Pвсас",
#     pump_out_cols[3]: "ПЭН-4 Pвсас",
#     pump_out_cols[4]: "ПЭН-5 Pвсас",
    
#     pump_mass_cols[0]: "ПЭН-1 Расход",
#     pump_mass_cols[1]: "ПЭН-2 Расход",
#     pump_mass_cols[2]: "ПЭН-3 Расход",
#     pump_mass_cols[3]: "ПЭН-4 Расход",
#     pump_mass_cols[4]: "ПЭН-5 Расход",
    
#     pump_temp_col: "Tпит.в",
    
#     pump_amper_cols[0]: "ПЭН-1 Сила тока",
#     pump_amper_cols[1]: "ПЭН-2 Сила тока",
#     pump_amper_cols[2]: "ПЭН-3 Сила тока",
#     pump_amper_cols[3]: "ПЭН-4 Сила тока",
#     pump_amper_cols[4]: "ПЭН-5 Сила тока",
    
#     pump_revs_cols[0]: "ПЭН-1 Част.вр.эл.дв.",
#     pump_revs_cols[1]: "ПЭН-2 Част.вр.эл.дв.",
#     pump_revs_cols[2]: "ПЭН-3 Част.вр.эл.дв.",
#     pump_revs_cols[3]: "ПЭН-4 Част.вр.эл.дв.",
#     pump_revs_cols[4]: "ПЭН-5 Част.вр.эл.дв.",
    
#     pump_volt_cols[0]: "Напряж. секц.BBA",
#     pump_volt_cols[1]: "Напряж. секц.BBB",
#     pump_volt_cols[2]: "Напряж. секц.BBC",
#     pump_volt_cols[3]: "Напряж. секц.BBD",
    
#     pump_turbine_col: "Мощность турб",
#     pump_reactor_col: "Мощность реакт",
# }, inplace = True)


# df.corr()

<h1>Нейросети</h1>

<p>Обучение на данных первого насоса</p>
<p>Выход - КПД, входы - давления, расход, темп, обороты, ток</p>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
#   plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Ошибка по модулю')  
  plt.plot(hist['epoch'], hist['mae'],
           label='Среднеквадратичная ошибка')
  plt.ylim([0,0.06])

<h3>Обучение с применением функции ReLU</h3>

In [ ]:
def build_and_train(pump_num, EPOCHS=3500, patience=500):
    pump_filtred_df = pd.DataFrame({
        'pump_in': filtred_data[f'pump_{pump_num}_in'],
        'pump_out': filtred_data[f'pump_{pump_num}_out'],
        'pump_mass': filtred_data[f'pump_{pump_num}_mass'],
        'pump_temp': filtred_data[f'pump_{pump_num}_temp'],
        'pump_revs': filtred_data[f'pump_{pump_num}_revs'],
        'pump_amper': filtred_data[f'pump_{pump_num}_amper'],
        'pump_eff': filtred_data[f'pump_{pump_num}_eff'],
    })

    train_dataset = pump_filtred_df.sample(frac=0.8,random_state=0)
    test_dataset = pump_filtred_df.drop(train_dataset.index)

    train_stats = train_dataset.describe()
    train_stats.pop("pump_eff")
    train_stats = train_stats.transpose()

    train_labels = train_dataset.pop('pump_eff')
    test_labels = test_dataset.pop('pump_eff')

    def norm(x):
      return (x - train_stats['mean']) / train_stats['std']

    normed_train_data = norm(train_dataset)
    normed_test_data = norm(test_dataset)


    def build_model():
      model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
      ])

      optimizer = tf.keras.optimizers.RMSprop(0.001)

      model.compile(loss='mse',
                    optimizer=optimizer,
                    metrics=['mae', 'mse'])
      return model

    model = build_model()
    model.summary()

    example_batch = normed_train_data[:10]
    example_result = model.predict(example_batch)

    model = build_model()

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    
    class PrintDot(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            print(f"Progress: {epoch}/{EPOCHS} {logs}", end='\r')

    history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                        validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

    plot_history(history)

    loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

    print("Testing set Mean Abs Error: {:5.5f}".format(mae))

    test_predictions = model.predict(normed_test_data).flatten()
    fig8 = plt.figure(figsize=(30, 5))
    ax8 = fig8.add_subplot(131)
    ax8.plot(test_predictions, color='blue', label='Предсказанное значение')
    ax8.plot(test_labels.reset_index()['pump_eff'], color='orange', label='Фактическое значение')
    ax8.set_ylabel(f'КПД ПЭН-{pump_num}')
    ax8.legend(loc='lower right')
    
    

In [ ]:
build_and_train(1) # обучение ПЭН-I

In [ ]:
build_and_train(2) # обучение ПЭН-II

In [ ]:
build_and_train(3) # обучение ПЭН-III

In [ ]:
build_and_train(4) # обучение ПЭН-IV

In [ ]:
build_and_train(5) # обучение ПЭН-V

<h3>Обучение с активационной функцией Sigmoid</h3>

In [ ]:
def build_and_train(pump_num, EPOCHS=3500, patience=500):
    pump_filtred_df = pd.DataFrame({
        'pump_in': filtred_data[f'pump_{pump_num}_in'],
        'pump_out': filtred_data[f'pump_{pump_num}_out'],
        'pump_mass': filtred_data[f'pump_{pump_num}_mass'],
        'pump_temp': filtred_data[f'pump_{pump_num}_temp'],
        'pump_revs': filtred_data[f'pump_{pump_num}_revs'],
        'pump_amper': filtred_data[f'pump_{pump_num}_amper'],
        'pump_eff': filtred_data[f'pump_{pump_num}_eff'],
    })

    train_dataset = pump_filtred_df.sample(frac=0.8,random_state=0)
    test_dataset = pump_filtred_df.drop(train_dataset.index)

    train_stats = train_dataset.describe()
    train_stats.pop("pump_eff")
    train_stats = train_stats.transpose()

    train_labels = train_dataset.pop('pump_eff')
    test_labels = test_dataset.pop('pump_eff')

    def norm(x):
      return (x - train_stats['mean']) / train_stats['std']

    normed_train_data = norm(train_dataset)
    normed_test_data = norm(test_dataset)


    def build_model():
      model = keras.Sequential([
        layers.Dense(64, activation='sigmoid', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation='sigmoid'),
        layers.Dense(1)
      ])

      optimizer = tf.keras.optimizers.RMSprop(0.001)

      model.compile(loss='mse',
                    optimizer=optimizer,
                    metrics=['mae', 'mse'])
      return model

    model = build_model()
    model.summary()

    example_batch = normed_train_data[:10]
    example_result = model.predict(example_batch)

    model = build_model()

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    
    class PrintDot(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            print(f"Progress: {epoch}/{EPOCHS} {logs}", end='\r')

    history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                        validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

    plot_history(history)

    loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

    print("Testing set Mean Abs Error: {:5.5f}".format(mae))

    test_predictions = model.predict(normed_test_data).flatten()
    fig8 = plt.figure(figsize=(30, 5))
    ax8 = fig8.add_subplot(131)
    ax8.plot(test_predictions, color='blue', label='Предсказанное значение')
    ax8.plot(test_labels.reset_index()['pump_eff'], color='orange', label='Фактическое значение')
    ax8.set_ylabel(f'КПД ПЭН-{pump_num}')
    ax8.legend(loc='lower right')
    
    

In [ ]:
build_and_train(1) # обучение ПЭН-I

In [ ]:
build_and_train(2) # обучение ПЭН-II

In [ ]:
build_and_train(3) # обучение ПЭН-III

In [ ]:
build_and_train(4) # обучение ПЭН-IV

In [ ]:
build_and_train(5) # обучение ПЭН-V

<h3>Обучение с активационной функцией tanh</h3>

In [ ]:
def build_and_train(pump_num, EPOCHS=3500, patience=500):
    pump_filtred_df = pd.DataFrame({
        'pump_in': filtred_data[f'pump_{pump_num}_in'],
        'pump_out': filtred_data[f'pump_{pump_num}_out'],
        'pump_mass': filtred_data[f'pump_{pump_num}_mass'],
        'pump_temp': filtred_data[f'pump_{pump_num}_temp'],
        'pump_revs': filtred_data[f'pump_{pump_num}_revs'],
        'pump_amper': filtred_data[f'pump_{pump_num}_amper'],
        'pump_eff': filtred_data[f'pump_{pump_num}_eff'],
    })

    train_dataset = pump_filtred_df.sample(frac=0.8,random_state=0)
    test_dataset = pump_filtred_df.drop(train_dataset.index)

    train_stats = train_dataset.describe()
    train_stats.pop("pump_eff")
    train_stats = train_stats.transpose()

    train_labels = train_dataset.pop('pump_eff')
    test_labels = test_dataset.pop('pump_eff')

    def norm(x):
      return (x - train_stats['mean']) / train_stats['std']

    normed_train_data = norm(train_dataset)
    normed_test_data = norm(test_dataset)


    def build_model():
      model = keras.Sequential([
        layers.Dense(64, activation='tanh', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation='tanh'),
        layers.Dense(1)
      ])

      optimizer = tf.keras.optimizers.RMSprop(0.001)

      model.compile(loss='mse',
                    optimizer=optimizer,
                    metrics=['mae', 'mse'])
      return model

    model = build_model()
    model.summary()

    example_batch = normed_train_data[:10]
    example_result = model.predict(example_batch)

    model = build_model()

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    
    class PrintDot(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            print(f"Progress: {epoch}/{EPOCHS} {logs}", end='\r')

    history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                        validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

    plot_history(history)

    loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

    print("Testing set Mean Abs Error: {:5.5f}".format(mae))

    test_predictions = model.predict(normed_test_data).flatten()
    fig8 = plt.figure(figsize=(30, 5))
    ax8 = fig8.add_subplot(131)
    ax8.plot(test_predictions, color='blue', label='Предсказанное значение')
    ax8.plot(test_labels.reset_index()['pump_eff'], color='orange', label='Фактическое значение')
    ax8.set_ylabel(f'КПД ПЭН-{pump_num}')
    ax8.legend(loc='lower right')
    
    

In [ ]:
build_and_train(1) # обучение ПЭН-I

In [ ]:
build_and_train(2) # обучение ПЭН-II

In [ ]:
build_and_train(3) # обучение ПЭН-III

In [ ]:
build_and_train(4) # обучение ПЭН-IV

In [ ]:
build_and_train(5) # обучение ПЭН-V

<h3>Обучение с использование LeakyReLU</h3>

In [ ]:
from tensorflow.keras.layers import LeakyReLU

def build_and_train(pump_num, EPOCHS=3500, patience=500):
    pump_filtred_df = pd.DataFrame({
        'pump_in': filtred_data[f'pump_{pump_num}_in'],
        'pump_out': filtred_data[f'pump_{pump_num}_out'],
        'pump_mass': filtred_data[f'pump_{pump_num}_mass'],
        'pump_temp': filtred_data[f'pump_{pump_num}_temp'],
        'pump_revs': filtred_data[f'pump_{pump_num}_revs'],
        'pump_amper': filtred_data[f'pump_{pump_num}_amper'],
        'pump_eff': filtred_data[f'pump_{pump_num}_eff'],
    })

    train_dataset = pump_filtred_df.sample(frac=0.8,random_state=0)
    test_dataset = pump_filtred_df.drop(train_dataset.index)

    train_stats = train_dataset.describe()
    train_stats.pop("pump_eff")
    train_stats = train_stats.transpose()

    train_labels = train_dataset.pop('pump_eff')
    test_labels = test_dataset.pop('pump_eff')

    def norm(x):
      return (x - train_stats['mean']) / train_stats['std']

    normed_train_data = norm(train_dataset)
    normed_test_data = norm(test_dataset)


    def build_model():
      model = keras.Sequential([
        LeakyReLU(alpha=0.1, input_shape=[len(train_dataset.keys())]),
        LeakyReLU(alpha=0.1),
        layers.Dense(1)
      ])

      optimizer = tf.keras.optimizers.RMSprop(0.001)

      model.compile(loss='mse',
                    optimizer=optimizer,
                    metrics=['mae', 'mse'])
      return model

    model = build_model()
    model.summary()

    example_batch = normed_train_data[:10]
    example_result = model.predict(example_batch)

    model = build_model()

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

    class PrintDot(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            print(f"Progress: {epoch}/{EPOCHS} {logs}", end='\r')
    
    history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                        validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

    plot_history(history)

    loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

    print("Testing set Mean Abs Error: {:5.5f}".format(mae))

    test_predictions = model.predict(normed_test_data).flatten()
    fig8 = plt.figure(figsize=(30, 5))
    ax8 = fig8.add_subplot(131)
    ax8.plot(test_predictions, color='blue', label='Предсказанное значение')
    ax8.plot(test_labels.reset_index()['pump_eff'], color='orange', label='Фактическое значение')
    ax8.set_ylabel(f'КПД ПЭН-{pump_num}')
    ax8.legend(loc='lower right')
    
    

In [ ]:
build_and_train(1) # обучение ПЭН-I

In [ ]:
build_and_train(2) # обучение ПЭН-Ii

In [ ]:
build_and_train(3) # обучение ПЭН-III

In [ ]:
build_and_train(4) # обучение ПЭН-IV

In [ ]:
build_and_train(5) # обучение ПЭН-V

<h3>Обучение с применением функции Softmax</h3>

In [ ]:
def build_and_train(pump_num, EPOCHS=3500, patience=500):
    pump_filtred_df = pd.DataFrame({
        'pump_in': filtred_data[f'pump_{pump_num}_in'],
        'pump_out': filtred_data[f'pump_{pump_num}_out'],
        'pump_mass': filtred_data[f'pump_{pump_num}_mass'],
        'pump_temp': filtred_data[f'pump_{pump_num}_temp'],
        'pump_revs': filtred_data[f'pump_{pump_num}_revs'],
        'pump_amper': filtred_data[f'pump_{pump_num}_amper'],
        'pump_eff': filtred_data[f'pump_{pump_num}_eff'],
    })

    train_dataset = pump_filtred_df.sample(frac=0.8,random_state=0)
    test_dataset = pump_filtred_df.drop(train_dataset.index)

    train_stats = train_dataset.describe()
    train_stats.pop("pump_eff")
    train_stats = train_stats.transpose()

    train_labels = train_dataset.pop('pump_eff')
    test_labels = test_dataset.pop('pump_eff')

    def norm(x):
      return (x - train_stats['mean']) / train_stats['std']

    normed_train_data = norm(train_dataset)
    normed_test_data = norm(test_dataset)


    def build_model():
      model = keras.Sequential([
        layers.Dense(64, activation='softmax', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation='softmax'),
        layers.Dense(1)
      ])

      optimizer = tf.keras.optimizers.RMSprop(0.001)

      model.compile(loss='mse',
                    optimizer=optimizer,
                    metrics=['mae', 'mse'])
      return model

    model = build_model()
    model.summary()

    example_batch = normed_train_data[:10]
    example_result = model.predict(example_batch)

    model = build_model()

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

    class PrintDot(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            print(f"Progress: {epoch}/{EPOCHS} {logs}", end='\r')
    
    history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                        validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

    plot_history(history)

    loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

    print("Testing set Mean Abs Error: {:5.5f}".format(mae))

    test_predictions = model.predict(normed_test_data).flatten()
    fig8 = plt.figure(figsize=(30, 5))
    ax8 = fig8.add_subplot(131)
    ax8.plot(test_predictions, color='blue', label='Предсказанное значение')
    ax8.plot(test_labels.reset_index()['pump_eff'], color='orange', label='Фактическое значение')
    ax8.set_ylabel(f'КПД ПЭН-{pump_num}')
    ax8.legend(loc='lower right')
    
    

In [ ]:
build_and_train(1) # обучение ПЭН-I

In [ ]:
build_and_train(2) # обучение ПЭН-II

In [ ]:
build_and_train(3) # обучение ПЭН-III

In [ ]:
build_and_train(4) # обучение ПЭН-IV

In [ ]:
build_and_train(5) # обучение ПЭН-V

In [ ]:
done_song = Audio('./success.wav', autoplay=True)
display(done_song)